In [57]:
import joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from Funcoes_Comuns import avaliar_modelo

In [58]:
def permutation_test_mae(y_true, y_pred_model1, y_pred_model2, n_permutations=10000):
    """
    Realiza um teste de permutação para comparar duas previsões de modelo usando o MAE.

    Args:
        y_true (array-like): Valores reais do conjunto de teste.
        y_pred_model1 (array-like): Previsões do Modelo 1 (ex: apenas socioeconômicos).
        y_pred_model2 (array-like): Previsões do Modelo 2 (ex: socioeconômicos + escolares).
        n_permutations (int): Número de permutações a serem realizadas.

    Returns:
        tuple: (p_value, observed_mae_diff)
               p_value: O p-valor do teste.
               observed_mae_diff: A diferença observada de MAE (MAE_model1 - MAE_model2).
                                  Um valor positivo indica que o Modelo 2 é melhor.
    """

    # 1. Calcular a diferença observada no MAE
    mae_model1 = mean_absolute_error(y_true, y_pred_model1)
    mae_model2 = mean_absolute_error(y_true, y_pred_model2)
    observed_mae_diff = mae_model1 - mae_model2 # Se positivo, significa que o Modelo 2 tem MAE menor

    permutation_diffs = []
    n_samples = len(y_true)

    # Obter os erros absolutos para cada ponto de dados e cada modelo
    abs_errors_model1 = np.abs(y_true - y_pred_model1)
    abs_errors_model2 = np.abs(y_true - y_pred_model2)

    for _ in range(n_permutations):
        # Cria uma máscara booleana para decidir quais erros absolutos "trocar"
        # entre os modelos para simular a hipótese nula (onde os erros vêm da mesma distribuição).
        swap_mask = np.random.rand(n_samples) < 0.5 # 50% de chance de trocar para cada ponto

        # Cria cópias dos erros absolutos para a permutação
        permuted_errors_model1 = np.copy(abs_errors_model1)
        permuted_errors_model2 = np.copy(abs_errors_model2)

        # Realiza a troca dos erros absolutos com base na máscara
        permuted_errors_model1[swap_mask] = abs_errors_model2[swap_mask]
        permuted_errors_model2[swap_mask] = abs_errors_model1[swap_mask]

        # Recalcula o MAE para os conjuntos de erros permutados
        perm_mae_model1 = np.mean(permuted_errors_model1)
        perm_mae_model2 = np.mean(permuted_errors_model2)

        # Calcula a diferença permutada
        perm_diff = perm_mae_model1 - perm_mae_model2
        permutation_diffs.append(perm_diff)

    permutation_diffs = np.array(permutation_diffs)

    # 4. Calcular o p-valor
    # Estamos testando se a diferença observada (MAE_model1 - MAE_model2) é POSITIVA
    # e significativa (ou seja, se MAE_model2 é significativamente menor que MAE_model1).
    # O p-valor é a proporção de diferenças permutadas que são maiores ou iguais à diferença observada.
    p_value = (np.sum(permutation_diffs >= observed_mae_diff) + 1) / (n_permutations + 1)

    return p_value, observed_mae_diff

In [59]:
# Ler base unificada do ENEM 2023 + Censo Escolar 2023
df_enem = pd.read_pickle('Bases\microdados_enem_censo_2023.pkl')

# 5 Variáveis alvo
# 40 Variáveis preditoras socioeconômicas ENEM
# 66 Variáveis preditoras escolares

In [60]:
colunas_alvo = ['NUM_NOTA_CH', 'NUM_NOTA_CN', 'NUM_NOTA_LC', 'NUM_NOTA_MT', 'NUM_NOTA_REDACAO']

colunas_socioeconomicas = [
    "BIN_Q001_DUMMY_H", "BIN_Q002_DUMMY_H", "BIN_Q018",
    "BIN_Q020", "BIN_Q021", "BIN_Q023", "BIN_Q025", "CAT_COR_RACA", "CAT_CO_MUNICIPIO_ESC", 
    "CAT_CO_UF_ESC", "CAT_DEPENDENCIA_ADM_ESC", "CAT_ENSINO", "CAT_ESCOLA", "CAT_ESTADO_CIVIL", "CAT_FAIXA_ETARIA",
    "CAT_LINGUA", "CAT_LOCALIZACAO_ESC", "CAT_NACIONALIDADE", "CAT_Q003", "CAT_Q004", "CAT_SEXO", "CAT_SIT_FUNC_ESC",
    "NUM_Q001", "NUM_Q002", "NUM_Q005", "NUM_Q006", "NUM_Q007", "NUM_Q008", "NUM_Q009", "NUM_Q010", "NUM_Q011",
    "NUM_Q012", "NUM_Q013", "NUM_Q014", "NUM_Q015", "NUM_Q016", "NUM_Q017", "NUM_Q019", "NUM_Q022", "NUM_Q024"
]

colunas_censo_escolar = [
    "CAT_MODE_CATEGORIA_ESCOLA_PRIVADA", "CAT_MODE_EXAME_SELECAO", "CAT_MODE_LOCALIZACAO_DIFERENCIADA", "CAT_MODE_OCUPACAO_GALPAO",
    "CAT_MODE_OCUPACAO_PREDIO_ESCOLAR", "CAT_MODE_ORGAO_REGIONAL", "CAT_MODE_PROPOSTA_PEDAGOGICA", "CAT_MODE_REGIAO",
    "CAT_MODE_TRATAMENTO_LIXO_INEXISTENTE",
    "NUM_MEAN_COMP_PORTATIL_ALUNO", "NUM_MEAN_DESKTOP_ALUNO", "NUM_MEAN_DOC_MED", "NUM_MEAN_EQUIP_LOUSA_DIGITAL",
    "NUM_MEAN_EQUIP_MULTIMIDIA", "NUM_MEAN_EQUIP_TV", "NUM_MEAN_MAT_MED", "NUM_MEAN_MAT_MED_INT",
    "NUM_MEAN_MAT_MED_NM", "NUM_MEAN_PROF_BIBLIOTECARIO", "NUM_MEAN_PROF_MONITORES", "NUM_MEAN_PROF_PEDAGOGIA",
    "NUM_MEAN_PROF_PSICOLOGO", "NUM_MEAN_SALAS_UTILIZADAS", "NUM_MEAN_TABLET_ALUNO", "NUM_MEAN_TUR_MED",
    "NUM_MEAN_TUR_MED_INT", "NUM_PERC_AGUA_INEXISTENTE", "NUM_PERC_AGUA_POTAVEL", "NUM_PERC_AREA_VERDE",
    "NUM_PERC_AUDITORIO", "NUM_PERC_BANHEIRO", "NUM_PERC_BIBLIOTECA", "NUM_PERC_ENERGIA_INEXISTENTE",
    "NUM_PERC_ESGOTO_INEXISTENTE", "NUM_PERC_INTERNET_ALUNOS", "NUM_PERC_INTERNET_APRENDIZAGEM", "NUM_PERC_LABORATORIO_CIENCIAS",
    "NUM_PERC_LABORATORIO_INFORMATICA", "NUM_PERC_LOCAL_FUNC_GALPAO", "NUM_PERC_LOCAL_FUNC_OUTROS", "NUM_PERC_LOCAL_FUNC_PREDIO_ESCOLAR",
    "NUM_PERC_LOCAL_FUNC_PRISIONAL_SOCIO", "NUM_PERC_LOCAL_FUNC_SALAS_OUTRA_ESC", "NUM_PERC_LOCAL_FUNC_SOCIOEDUCATIVO", "NUM_PERC_LOCAL_FUNC_UNID_PRISIONAL",
    "NUM_PERC_MANT_ESCOLA_PRIVADA_EMP", "NUM_PERC_MANT_ESCOLA_PRIVADA_ONG", "NUM_PERC_MANT_ESCOLA_PRIVADA_OSCIP", "NUM_PERC_MANT_ESCOLA_PRIVADA_SIND",
    "NUM_PERC_MANT_ESCOLA_PRIVADA_SIST_S", "NUM_PERC_MANT_ESCOLA_PRIVADA_S_FINS", "NUM_PERC_MANT_ESCOLA_PRIV_ONG_OSCIP", "NUM_PERC_MATERIAL_PED_NENHUM",
    "NUM_PERC_MEDIACAO_EAD", "NUM_PERC_MEDIACAO_PRESENCIAL", "NUM_PERC_MEDIACAO_SEMIPRESENCIAL", "NUM_PERC_PISCINA",
    "NUM_PERC_PODER_PUBLICO_PARCERIA", "NUM_PERC_PROF_TEC", "NUM_PERC_QUADRA_ESPORTES", "NUM_PERC_SALA_ATELIE_ARTES",
    "NUM_PERC_SALA_MUSICA_CORAL", "NUM_PERC_VINCULO_OUTRO_ORGAO", "NUM_PERC_VINCULO_SECRETARIA_EDUCACAO", "NUM_PERC_VINCULO_SECRETARIA_SAUDE",
    "NUM_PERC_VINCULO_SEGURANCA_PUBLICA"
]

In [61]:
# separar em treino e teste
X = df_enem.drop(columns=colunas_alvo)
y = df_enem['NUM_NOTA_CH']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [62]:
# Converter colunas inteiras para float
X_test = X_test.astype({col: 'float' for col in X_test.select_dtypes('int').columns})

In [63]:
# Selecionar apenas as colunas socioeconômicas
X_test_socio = X_test[colunas_socioeconomicas]

In [64]:
# Carregar modelo treinado socioeconômico
with open('Modelos\modelo_lgbm_bayes.pkl', 'rb') as file:
    modelo_socio = joblib.load(file)

# Carregar modelo treinado socioeconômico + escolar
with open('Modelos\modelo_lgbm_bayes_censo_enem_NUM_NOTA_CH.pkl', 'rb') as file:
    modelo_escolar = joblib.load(file)

In [65]:
# Aplicar o modelo nos dados de teste
y_pred_socio = modelo_socio.predict(X_test_socio)

In [66]:
# Avaliação grupo teste
avaliar_modelo(y_test, y_pred_socio, "teste")

MAE (teste): 53.5000
RMSE (teste): 67.9310
R2 (teste): 0.3527


In [ ]:
# Temporario até ajustar o modelos treinado
# Alterar as colunas: "CAT_DEPENDENCIA_ADM_ESC", "CAT_CO_MUNICIPIO_ESC" e "CAT_LOCALIZACAO_ESC" para int
X_test['CAT_DEPENDENCIA_ADM_ESC'] = X_test['CAT_DEPENDENCIA_ADM_ESC'].astype(int)
X_test['CAT_CO_MUNICIPIO_ESC'] = X_test['CAT_CO_MUNICIPIO_ESC'].astype(int)
X_test['CAT_LOCALIZACAO_ESC'] = X_test['CAT_LOCALIZACAO_ESC'].astype(int)

# Converter colunas inteiras para float
X_test = X_test.astype({col: 'float' for col in X_test.select_dtypes('int').columns})

In [68]:
y_pred_socio_escolar = modelo_escolar.predict(X_test)

In [69]:
# Avaliação grupo teste
avaliar_modelo(y_test, y_pred_socio_escolar, "teste")

MAE (teste): 54.7993
RMSE (teste): 69.4808
R2 (teste): 0.3228


In [70]:
# Suponha que você já tenha:
# y_test: seus valores reais do conjunto de teste (com 143.000 elementos)
# y_pred_socio: previsões do seu modelo apenas com dados socioeconômicos
# y_pred_socio_escolar: previsões do seu modelo com dados socioeconômicos + escolares

print("Executando Teste de Permutação para MAE...")
p_value, obs_diff = permutation_test_mae(
    y_test,
    y_pred_socio,
    y_pred_socio_escolar,
    n_permutations=10000 # Você pode aumentar este número para maior precisão
)

# Exibir os resultados
mae_m1 = mean_absolute_error(y_test, y_pred_socio)
mae_m2 = mean_absolute_error(y_test, y_pred_socio_escolar)

print(f"\nMAE do Modelo Socioeconômico: {mae_m1:.4f}")
print(f"MAE do Modelo Socioeconômico + Escola: {mae_m2:.4f}")
print(f"Diferença observada (MAE_socio - MAE_socio_escola): {obs_diff:.4f}")
print(f"P-valor: {p_value:.4f}")

# Interpretação do resultado
alpha = 0.05 # Nível de significância comum
if p_value < alpha:
    print(f"\nCom p-valor {p_value:.4f} < {alpha}, rejeitamos a hipótese nula.")
    print("A melhora observada no MAE com os dados escolares é estatisticamente significativa.")
else:
    print(f"\nCom p-valor {p_value:.4f} >= {alpha}, falhamos em rejeitar a hipótese nula.")
    print("A melhora observada no MAE NÃO é estatisticamente significativa (pode ser devido ao acaso).")

Executando Teste de Permutação para MAE...

MAE do Modelo Socioeconômico: 53.5000
MAE do Modelo Socioeconômico + Escola: 54.7993
Diferença observada (MAE_socio - MAE_socio_escola): -1.2992
P-valor: 1.0000

Com p-valor 1.0000 >= 0.05, falhamos em rejeitar a hipótese nula.
A melhora observada no MAE NÃO é estatisticamente significativa (pode ser devido ao acaso).
